In [8]:
f = "/home/nikola/projects/parlaspeech/ParlaSpeech-HR.v1.0.jsonl"

import pandas as pd
n = pd.read_json(f, orient="records", lines=True)

In [9]:
n.columns

Index(['path', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times'],
      dtype='object')

In [10]:
n.head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times
0,j8IDWUzSlX0_3942.4-3958.56.wav,"11 7 2018 - 2. dio, 8. sjednica, 9. saziv [j8I...",3942.4,3958.56,"[Povrat, koji, se, rade, na, pojedinim, područ...","[0, 0.8300000000000001, 1.08, 2.19, 3.09, 4.31...","[povrat, koji, se, rade, na, pojedinim, područ...","[0, 0.8300000000000001, 1.08, 2.19, 3.09, 4.31..."
1,s7v5eLALCvY_4260.3-4276.1.wav,"27 3 2020 - 16. sjednica, 9. saziv [s7v5eLALCv...",4260.3,4276.10,"[a, isto, tako, i, izvan, RH, u, zemljama, EU,...","[0, 0.18, 0.39, 0.98, 2.13, 2.41, 3.4, 3.47, 4...","[a, isto, tako, i, izvan, erha, u, zemljama, e...","[0, 0.18, 0.39, 0.98, 2.13, 2.41, 3.4, 3.47, 4..."


In [12]:
ps = pd.read_csv("parlaspeech.csv")

In [25]:
ps_files = set(ps.hashname.values.tolist())
n_files = set(n.path.values.tolist())

len(ps_files.difference(n_files))

40463

In [26]:
import os
actual_files = set(os.listdir("selection_16000_mono/"))
len(actual_files.difference(n_files))

26428

In [18]:
s = pd.read_csv("selection.csv")

In [27]:
selection_files = set(s.hashname.values.tolist())
len(selection_files.difference(n_files))

26428

In [22]:
(s.sim < 0.8).sum()

26428

In [23]:
n.shape

(407079, 8)

In [24]:
s.shape

(433507, 22)

In [29]:
s["is_in_n"] = s.hashname.isin(n.path)

In [30]:
s.is_in_n.sum()

407079

In [36]:
s.loc[s.is_in_n, :].drop(columns="is_in_n").to_csv("selection.csv", index=False)
s.loc[s.is_in_n, :].drop(columns="is_in_n").to_json("selection.json", orient = "records", lines=True)

In [35]:


from concurrent.futures import ProcessPoolExecutor
import os

def delete(file):
    os.system(f"rm selection_16000_mono/{file}")
with ProcessPoolExecutor(max_workers=32) as executor:
    executor.map(
        delete,
        s.loc[~s.is_in_n, "hashname"])